In [1]:
import pandas as pd
import numpy as np


In [2]:
df1 = pd.read_csv('data.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)

df2 = pd.read_csv('data.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)


C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df1.label[df1.label == 1]="acpt"
df1.label[df1.label == 0]="unac"

df2.label[df2.label == 1]="acpt"
df2.label[df2.label == 0]="unac"

C:\Users\moshi\AppData\Local\Temp/ipykernel_14944/3898086040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.label[df1.label == 1]="acpt"
C:\Users\moshi\AppData\Local\Temp/ipykernel_14944/3898086040.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.label[df2.label == 1]="acpt"


In [4]:
# taking acceptable and deleting unacceptable
df3 = df1[~df1.label.str.contains('acpt')]

In [5]:
# randomly picking 250 rows
df3 = df3.sample(n=250, replace=True)

In [6]:
df3["label"].value_counts()

unac    250
Name: label, dtype: int64

In [7]:
# taking unacceptable and deleting unacceptable
df4 = df2[~df2.label.str.contains("unac")]

In [8]:
# randomly picking 250 rows
df4 = df4.sample(n=250, replace=True)

In [9]:
df4["label"].value_counts()

acpt    250
Name: label, dtype: int64

In [10]:
#concating acceptable and unacceptable
frames = [df3, df4]

temp = pd.concat(frames)

In [11]:
temp["label"].value_counts()

unac    250
acpt    250
Name: label, dtype: int64

In [12]:
temp.head()

,label,text
7429,unac,"Mary won't have been eating cake, but John."
6322,unac,Heidi believes Bill's description of herself.
1440,unac,Did that he played the piano surprise you?
270,unac,You give every senator enough opportunity and ...
2819,unac,The cloth swatted the fly.


In [13]:
temp.tail()

,label,text
1380,acpt,Which bed did Tom sleep on?
1456,acpt,Her efficient looking up of the answer pleased...
8235,acpt,I asked which city which king invaded.
3083,acpt,Ellen said something to Helen.
3768,acpt,A review copy of the book was sent to her by t...


In [14]:
temp["label"].value_counts()

unac    250
acpt    250
Name: label, dtype: int64

In [15]:
text = temp['text'].values.tolist()
emotion= temp['label'].tolist()

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text , emotion, test_size=.2, random_state=42)

In [17]:
import ktrain
from ktrain import text
MODEL_NAME = 'prajjwal1/bert-tiny'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [18]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 8
	95percentile : 16
	99percentile : 21


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 8
	95percentile : 15
	99percentile : 19


In [19]:
model = t.get_classifier()

C:\Users\moshi\anaconda3\lib\site-packages\ktrain\text\preprocessor.py:1069: UserWarning: Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  warnings.warn('Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. ' +\


In [20]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [21]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
67/67 [==============================] - 24s 288ms/step - loss: 0.6973 - accuracy: 0.5225 - val_loss: 0.6989 - val_accuracy: 0.4800
Epoch 2/5
67/67 [==============================] - 19s 282ms/step - loss: 0.6973 - accuracy: 0.5050 - val_loss: 0.6969 - val_accuracy: 0.5000
Epoch 3/5
67/67 [==============================] - 19s 276ms/step - loss: 0.6904 - accuracy: 0.5150 - val_loss: 0.6960 - val_accuracy: 0.5300
Epoch 4/5
67/67 [==============================] - 18s 269ms/step - loss: 0.6881 - accuracy: 0.5325 - val_loss: 0.6948 - val_accuracy: 0.5100
Epoch 5/5
67/67 [==============================] - 18s 270ms/step - loss: 0.6808 - accuracy: 0.5600 - val_loss: 0.6938 - val_accuracy: 0.4800


In [22]:
class_names=["acpt","unac"]

In [23]:
learner.validate(val_data=(val), class_names=class_names)

              precision    recall  f1-score   support

        acpt       0.55      0.22      0.32        54
        unac       0.46      0.78      0.58        46

    accuracy                           0.48       100
   macro avg       0.50      0.50      0.45       100
weighted avg       0.51      0.48      0.44       100



array([[12, 42],
       [10, 36]], dtype=int64)